In [1]:
# Load packages
import numpy as np
import pandas as pd
from numba import jit,float64
from scipy.optimize import minimize

In [2]:
# Load data
data = pd.read_csv('UnitaryData.csv')
pd_lag = np.array(data['d.p'])

# Calculate terciles for pd ratio
tercile_1 = np.quantile(pd_lag,1./3)
tercile_2 = np.quantile(pd_lag,2./3)

# Calculate indicator based on today's pd ratio
pd_lag_indicator = np.array([pd_lag <= tercile_1,(pd_lag <= tercile_2) & (pd_lag > tercile_1),pd_lag > tercile_2]).T

# Calculate indicator for tomorrow's pd ratio
pd_indicator = pd_lag_indicator[1:]

# Drop last row since we do not have tomorrow's pd ratio at that point
pd_lag_indicator = pd_lag_indicator[:-1]
X = np.array(data[['Rf','Rm-Rf','SMB','HML']])[:-1]
f 
g = np.array(data['log.RW'])[:-1]

## 7 Incorporating Conditional Information

### 7.1 Approach I
\begin{equation}
\min_{M\geq0}{\mathbb E}\left[Mg(X)\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M \log M\right] \leq \kappa, \\
&{\mathbb E}\left[B^j M f(X)\right] = 0, \\
&{\mathbb E}\left[M\right] = 1.
\end{align*}


### 7.2 Approach II
\begin{equation}
\min_{M\geq0}{\mathbb E}\left[Mg(X)\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M \log M\right] \leq \kappa, \\
&{\mathbb E}\left[B^j M {f(X)}\right] = 0, \\
&{\mathbb E}\left[B^j M-B^j\right] = 0.
\end{align*}

### 7.3 Approach III

#### Basic problem:

\begin{equation}
\min_{M\geq0}{\mathbb E}\left[Mg(X)\mid\mathfrak{J}\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M \log M\mid\mathfrak{J}\right] \leq \kappa, \\
&{\mathbb E}\left[M  f(X)\mid\mathfrak{J}\right] = 0, \\
&{\mathbb E}\left[M\mid\mathfrak{J}\right] = 1.
\end{align*}

#### Dual problem:
For computational purposes, we solve the dual problem after minimizing over $M$.  

\begin{equation*}
\max_{\xi \ge 0, \hat{\lambda}}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ g(X) + \hat{\lambda} \cdot f(X) \right] \right)\mid\mathfrak{F}\right]  -  \xi \kappa  
\end{equation*}

$\hat{\lambda}$ and $\xi$ are multipliers on the moment condition and relative entropy constraints. 

## 8 Intertemporal Divergence Constraints

### Proposition 8.6
Problem 8.4 can be solved by finding the solution to:

\begin{equation}
\epsilon = \min_\hat{\lambda}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{F}_0\right)
\end{equation}

*where*
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

Denote $e_0^*$, $e_1^*$, $\hat{\lambda}^*$ as the solution to the above optimization problem. The implied solution for the probablity distortion is:

\begin{equation}
M_1^* = \frac{\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}^*(Z_0)\cdot f(X_1)\right]e_1^*}{\epsilon^*e_0^*}
\end{equation}

In [171]:
def objective(λ):
    selector = pd_lag_indicator[:,state-1]
    term_1 = -g[selector]/ξ
    term_2 = f[selector]@λ
    term_3 = np.log(pd_indicator[selector]@e)
    x = term_1 + term_2 + term_3
    # use "max trick" to improve accuracy
    a = x.max()
    # log_E_exp(x)
    return np.log(np.sum(np.exp(x-a))) + a

def min_objective():
    model = minimize(objective, 
                     np.ones(f.shape[1]), 
                     method='L-BFGS-B',
                     tol=1e-10,
                     options={'maxiter': 1000})
    v = np.exp(model.fun)/np.sum(pd_lag_indicator[:,state-1])
    λ = model.x
    return v,λ

def iteration():
    # set global variables
    global state
    global e
    
    # initial error
    error = 1.
    # count times
    count = 0

    while error > 1e-10:
        if count == 0:
            # initial guess for e
            e = np.array([1,1,1])
            # placeholder for v
            v = np.zeros(3)        
        for k in [1,2,3]:
            state = k
            v[state-1],λ = min_objective()
        # update e and ϵ
        e_old = e
        ϵ = v[0]
        e = v/v[0]
        error = np.max(np.abs(e - e_old))
        count += 1
        
    return ϵ,e,λ,count

In [174]:
ξ = 1.

time_start = time.time() 
ϵ,e,λ,count = iteration()

print("--- %s seconds ---" % (round(time.time()-time_start,4)))
print("--- %s iterations ---" % count)
print("--- ϵ: %s ---" % ϵ)
print("--- e: %s ---" % e)

--- 1.5485 seconds ---
--- 235 iterations ---
--- ϵ: 0.9662646500347174 ---
--- e: [1.         0.44191812 0.19224314] ---


In [175]:
# Calculate M
M = 1./ϵ * np.exp(-g/ξ+f@λ) * (pd_indicator@e) / (pd_lag_indicator@e)

# Check 1: E[M|state k] = 1
# np.average(M[pd_lag_indicator[:,0]])
# np.average(M[pd_lag_indicator[:,1]])
np.average(M[pd_lag_indicator[:,2]])

0.9999999996547466

In [177]:
len(data)/3

82.66666666666667